# Class Demos from 20251024 - Oct 24, 2025

Demoing various things, including:

- XPath & functionality in python using lxml
- MODS and searching through MODS (encoded in XML)
- working with LOC 'API' to extract collection content